#### Example Socrata Module for City of Chicago Data. https://dev.socrata.com/foundry/data.cityofchicago.org/6zsd-86xi
* I'll move the client info to a config file later. 
* I don't use the sodapy package, the code is more efficient, but I'm trying to demonstrate that I'm learning the DS_Bootcamp code

In [1]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import warnings;
warnings.filterwarnings('ignore');

# import pandas as pd
from sodapy import Socrata

MyAppToken = 'PoLZLhceSUgObx1Disd6WIQS1'

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofchicago.org", None)

# Example authenticated client (needed for non-public datasets):
client = Socrata('data.cityofchicago.org',
                 MyAppToken,
                 username="stephen.monteiro@northwestern.edu",
                 password="FackeTestPw0rd")

#### Importing Modules & Data
* With more time and a better coder, you could loop through each API call, hence why I define year as a list. Optimistic I'll return to it one day. 

In [2]:
#Dependencies
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pprint as pp
import os
import csv
year = [2002, 2010, 2018]

In [3]:
fbi_codex = pd.read_csv('fbi_codex.csv')
fbi_index = fbi_codex[['fbi_Crime_type','fbi_code_key','ds_fbi_group']]
fbi_index = fbi_index.dropna()
gent_codex = pd.read_csv('uic_gent_index.csv')
gent_concat = gent_codex["Type"].str.split(" - ", n = 1, expand = True) 
gent_codex["Classification"] = gent_concat[1]
gent_codex["Class_Code"] = gent_concat[0]
gent_fin = gent_codex.rename(columns={"No.": "community_area"})
gent_fin
display(fbi_index.head())
display(gent_fin.head())

,fbi_Crime_type,fbi_code_key,ds_fbi_group
0,Aggravated Assault,04A,Violent
1,Aggravated Battery,04B,Violent
2,Arson,9,Nonviolent
3,Burglary,5,Nonviolent
4,Criminal Sexual Abuse,17,Nonviolent


,community_area,Neighborhood,1970,1980,1990,2000,2010,1970-2010,Change,Type,Classification,Class_Code
0,77,Edgewater,7,9,9,7,9,2,No Change,1 - Upper Class,Upper Class,1
1,9,Edison Park,10,9,11,11,11,1,No Change,1 - Upper Class,Upper Class,1
2,12,Forest Glen,11,11,11,11,9,-2,No Change,1 - Upper Class,Upper Class,1
3,4,Lincoln Square,9,7,5,11,11,2,No Change,1 - Upper Class,Upper Class,1
4,13,North Park,9,11,11,11,11,2,No Change,1 - Upper Class,Upper Class,1


#### API Call for Start Year
* Currently using 2002 rather than 2001, because 2001 data lacks 'community_area' for most reported crimes. With more time, you could use loops and conditions on 'latitude' and 'longitude' to fill 'community area'.
* Export the data to CSV so that there is a reference file in case original data source is ever corrupted, or non-responsive to API calls.
* For more on the City of Chicago Crime Data: https://dev.socrata.com/foundry/data.cityofchicago.org/6zsd-86xi

* FIGURE OUT: how to remove decimal from neighborhood

In [4]:
results = client.get("6zsd-86xi", year=2002, limit=5000)
results_df = pd.DataFrame.from_records(results) # Convert to pandas DataFrame # results_df.head()
start_results = results_df[['community_area','fbi_code','date','description','latitude','longitude','primary_type','year']]
start_results['community_area'] = pd.to_numeric(start_results['community_area'], errors='coerce')
start_format = start_results.sort_values('community_area')
start_format.to_csv('report_reference_data/start_tfile.csv')    
start_format.head(10)


,community_area,fbi_code,date,description,latitude,longitude,primary_type,year
634,1.0,07,2002-05-22T07:10:00.000,THEFT/RECOVERY: AUTOMOBILE,NaN,NaN,MOTOR VEHICLE THEFT,2002
1083,1.0,18,2002-09-18T14:30:00.000,MANU/DELIVER:CRACK,NaN,NaN,NARCOTICS,2002
803,1.0,06,2002-07-16T19:00:00.000,$500 AND UNDER,NaN,NaN,THEFT,2002
1238,1.0,18,2002-08-09T14:30:40.000,POSS: CANNABIS MORE THAN 30GMS,NaN,NaN,NARCOTICS,2002
1428,1.0,26,2002-11-02T16:51:12.000,TO LAND,NaN,NaN,CRIMINAL TRESPASS,2002
1500,1.0,06,2002-09-02T00:00:00.000,FINANCIAL ID THEFT: OVER $300,42.016524687,-87.675285207,THEFT,2002
863,1.0,18,2002-07-30T12:06:42.000,MANU/DEL:CANNABIS OVER 10 GMS,NaN,NaN,NARCOTICS,2002
1440,1.0,17,2002-11-04T15:00:00.000,AGG CRIMINAL SEXUAL ABUSE,NaN,NaN,SEX OFFENSE,2002
656,1.0,08B,2002-06-07T20:30:00.000,SIMPLE,NaN,NaN,BATTERY,2002
764,1.0,06,2002-07-08T18:45:00.000,OVER $500,NaN,NaN,THEFT,2002


#### Combining the Crime Data with our Research Indexes (Gentrification & FBI Crime Category)

,community_area,fbi_code,date,description,latitude,longitude,primary_type,year,fbi_Crime_type,fbi_code_key,...,1970,1980,1990,2000,2010,1970-2010,Change,Type,Classification,Class_Code
0,1.0,18,2002-09-18T14:30:00.000,MANU/DELIVER:CRACK,NaN,NaN,NARCOTICS,2002,Drug Abuse,18,...,9,9,3,1,3,-6,Negative,7 - Mild Decline,Mild Decline,7
1,1.0,18,2002-08-09T14:30:40.000,POSS: CANNABIS MORE THAN 30GMS,NaN,NaN,NARCOTICS,2002,Drug Abuse,18,...,9,9,3,1,3,-6,Negative,7 - Mild Decline,Mild Decline,7
2,1.0,18,2002-07-30T12:06:42.000,MANU/DEL:CANNABIS OVER 10 GMS,NaN,NaN,NARCOTICS,2002,Drug Abuse,18,...,9,9,3,1,3,-6,Negative,7 - Mild Decline,Mild Decline,7
3,1.0,18,2002-08-02T03:25:58.000,POSS: CANNABIS 30GMS OR LESS,NaN,NaN,NARCOTICS,2002,Drug Abuse,18,...,9,9,3,1,3,-6,Negative,7 - Mild Decline,Mild Decline,7
4,1.0,18,2002-07-31T11:00:00.000,MANU/DELIVER:CRACK,NaN,NaN,NARCOTICS,2002,Drug Abuse,18,...,9,9,3,1,3,-6,Negative,7 - Mild Decline,Mild Decline,7
5,1.0,18,2002-08-17T19:05:00.000,POSS: CANNABIS 30GMS OR LESS,NaN,NaN,NARCOTICS,2002,Drug Abuse,18,...,9,9,3,1,3,-6,Negative,7 - Mild Decline,Mild Decline,7
6,1.0,18,2002-07-10T14:22:54.000,POSS: CANNABIS 30GMS OR LESS,NaN,NaN,NARCOTICS,2002,Drug Abuse,18,...,9,9,3,1,3,-6,Negative,7 - Mild Decline,Mild Decline,7
7,1.0,26,2002-11-02T16:51:12.000,TO LAND,NaN,NaN,CRIMINAL TRESPASS,2002,Misc Non-Index Offense,26,...,9,9,3,1,3,-6,Negative,7 - Mild Decline,Mild Decline,7
8,1.0,26,2002-08-28T08:25:00.000,FALSE/STOLEN/ALTERED TRP,NaN,NaN,OTHER OFFENSE,2002,Misc Non-Index Offense,26,...,9,9,3,1,3,-6,Negative,7 - Mild Decline,Mild Decline,7
9,1.0,17,2002-11-04T15:00:00.000,AGG CRIMINAL SEXUAL ABUSE,NaN,NaN,SEX OFFENSE,2002,Criminal Sexual Abuse,17,...,9,9,3,1,3,-6,Negative,7 - Mild Decline,Mild Decline,7


In [6]:
b_crimecat_cca_clean = b_crimecat_cca[['community_area', 'Neighborhood', 'fbi_code','fbi_Crime_type', 'ds_fbi_group', 'Classification', 'Class_Code',]]
b_crimecat_cca_group = b_crimecat_cca_clean.groupby(by=['ds_fbi_group','Classification']).count() #relevant_results.sort_values('fbi_code')
crime_reset = b_crimecat_cca_group.reset_index()
print("Count of FBI Crime Categories by Gentrification Pathway")
crime_reset.drop(columns=['community_area', 'Neighborhood', 'fbi_code','fbi_Crime_type',])
crime_reset.pivot(index='Classification', columns='ds_fbi_group', values='Class_Code')


Count of FBI Crime Categories by Gentrification Pathway


ds_fbi_group,Nonviolent,Violent,White Collar
Classification,,,
Extreme Poverty,52.0,31.0,4.0
Gentrification,71.0,32.0,11.0
Middle Class,17.0,11.0,7.0
Mild Decline,23.0,5.0,1.0
Moderate Decline,53.0,16.0,4.0
Not Gentrification,11.0,8.0,2.0
Poverty,14.0,5.0,NaN
Serious Decline,86.0,31.0,5.0
Upper Class,6.0,1.0,1.0


In [7]:
b_crimecat_cca_clean['Class_Code'] = pd.to_numeric(b_crimecat_cca_clean['Class_Code'], errors='coerce') # b_crimecat_cca_clean.dtypes
ng1 = b_crimecat_cca_clean.loc[b_crimecat_cca_clean['Class_Code'] < 3]
ng2 = b_crimecat_cca_clean.loc[b_crimecat_cca_clean['Class_Code'] == 5]
g6 = b_crimecat_cca_clean.loc[b_crimecat_cca_clean['Class_Code'] == 6]
combo = [ng1, ng2]
nGentrified = pd.concat(combo)
nGentrified["DS_Subset"] = "Not Gentrified"
g6["DS_Subset"] = "Gentrified"
neighborhodtype_bycrime = [nGentrified, g6]
neighbors = pd.concat(neighborhodtype_bycrime)
neighbors_reset = neighbors.groupby(by=['DS_Subset','ds_fbi_group']).count()
neighbors_dropem = neighbors_reset.reset_index()
neighbors_dropem.drop(columns=['Classification','community_area', 'Neighborhood', 'fbi_code','fbi_Crime_type',], inplace=True)
print("Count of FBI Crime Categories by Gentrification Pathway")
neighbors_dropem.pivot(index='ds_fbi_group', columns='DS_Subset', values='Class_Code')


Count of FBI Crime Categories by Gentrification Pathway


DS_Subset,Gentrified,Not Gentrified
ds_fbi_group,,
Nonviolent,71,34
Violent,32,20
White Collar,11,10


In [8]:
start_group = start_format.groupby(by=['community_area', 'fbi_code']).count() #relevant_results.sort_values('fbi_code'
start_group


date  description  latitude  longitude  primary_type  \
community_area fbi_code                                                         
1.0            03           2            2         0          0             2   
               04A          1            1         0          0             1   
               06           7            7         2          2             7   
               07           1            1         0          0             1   
               08B          3            3         0          0             3   
               11           1            1         0          0             1   
               14           1            1         1          1             1   
               17           1            1         0          0             1   
               18           7            7         0          0             7   
               26           2            2         0          0             2   
2.0            03           2            2         0          0             2   
               06           8            8         1          1             8   
               07           3            3         0          0             3   
               08B          2            2         0          0             2   
               11           1            1         0          0             1   
3.0            01A          2            2         2          2             2   
               03           1            1         0          0             1   
               04A          1            1         1          1             1   
               06           7            7         1          1             7   
               07           2            2         0          0             2   
               08B          2            2         0          0             2   
               11           1            1         0          0             1   
               14           1            1         0          0             1   
               18           4            4         0          0             4   
               26           4            4         0          0             4   
4.0            03           1            1         0          0             1   
               06           2            2         0          0             2   
               07           1            1         0          0             1   
               11           1            1         0          0             1   
               16           1            1         0          0             1   
...                       ...          ...       ...        ...           ...   
70.0           20           1            1         0          0             1   
71.0           01A          1            1         1          1             1   
               02           1            1         0          0             1   
               03           2            2         0          0             2   
               06           5            5         4          4             5   
               07           3            3         0          0             3   
               08B          1            1         0          0             1   
               11           1            1         1          1             1   
               13           1            1         0          0             1   
               15           2            2         0          0             2   
               20           1            1         0          0             1   
               26           1            1         1          1             1   
72.0           06           1            1         1          1             1   
               07           1            1         0          0             1   
73.0           01A          2            2         2          2             2   
               02           1            1         0          0             1   
               06           2            2         2    

In [9]:
midresults = client.get("6zsd-86xi", year=2010, limit=5000)
midresults_df = pd.DataFrame.from_records(midresults) # Convert to pandas DataFrame # results_df.head()
mid_results = midresults_df[['community_area','fbi_code','date','description','latitude','longitude','primary_type','year']]
mid_results.groupby(by=['fbi_code']).count() #relevant_results.sort_values('fbi_code')
mid_results.sort_values(['community_area'])
mid_results.to_csv('report_reference_data/mid_tfile.csv')    
mid_results

,community_area,fbi_code,date,description,latitude,longitude,primary_type,year
0,71,02,2010-01-01T00:01:00.000,PREDATORY,NaN,NaN,CRIM SEXUAL ASSAULT,2010
1,41,11,2010-01-01T12:00:00.000,FINANCIAL IDENTITY THEFT $300 AND UNDER,NaN,NaN,DECEPTIVE PRACTICE,2010
2,11,02,2010-01-01T00:00:00.000,SEX ASSLT OF CHILD BY FAM MBR,NaN,NaN,OFFENSE INVOLVING CHILDREN,2010
3,67,11,2010-01-11T18:20:00.000,FINANCIAL IDENTITY THEFT OVER $ 300,NaN,NaN,DECEPTIVE PRACTICE,2010
4,66,02,2010-05-01T00:00:00.000,AGGRAVATED: OTHER,NaN,NaN,CRIM SEXUAL ASSAULT,2010
5,69,11,2010-01-01T00:01:00.000,FINANCIAL IDENTITY THEFT $300 AND UNDER,NaN,NaN,DECEPTIVE PRACTICE,2010
6,38,11,2010-07-01T08:00:00.000,FINANCIAL IDENTITY THEFT OVER $ 300,NaN,NaN,DECEPTIVE PRACTICE,2010
7,77,02,2010-09-28T00:01:00.000,NON-AGGRAVATED,NaN,NaN,CRIM SEXUAL ASSAULT,2010
8,60,11,2010-01-26T09:00:00.000,FINAN EXPLOIT-ELDERLY/DISABLED,NaN,NaN,DECEPTIVE PRACTICE,2010
9,14,17,2010-09-18T12:00:00.000,AGG CRIMINAL SEXUAL ABUSE,NaN,NaN,SEX OFFENSE,2010


In [10]:
closeresults = client.get("6zsd-86xi", year=2018, limit=5000)
closeresults_df = pd.DataFrame.from_records(closeresults)                    # Convert to pandas DataFrame # results_df.head()
close_results = closeresults_df[['community_area','fbi_code','date','description', 'latitude','longitude','primary_type','year']]
close_results.groupby(by=['fbi_code']).count() #relevant_results.sort_values('fbi_code')
close_results.to_csv('report_reference_data/close_tfile.csv')    
close_results


,community_area,fbi_code,date,description,latitude,longitude,primary_type,year
0,46,11,2018-12-29T00:00:00.000,ILLEGAL USE CASH CARD,NaN,NaN,DECEPTIVE PRACTICE,2018
1,61,15,2018-07-27T19:44:31.000,UNLAWFUL POSS OF HANDGUN,NaN,NaN,WEAPONS VIOLATION,2018
2,23,06,2018-12-31T21:00:00.000,FROM BUILDING,NaN,NaN,THEFT,2018
3,67,06,2018-06-01T13:42:00.000,$500 AND UNDER,NaN,NaN,THEFT,2018
4,21,11,2018-09-24T00:01:00.000,FRAUD OR CONFIDENCE GAME,NaN,NaN,DECEPTIVE PRACTICE,2018
5,33,22,2018-11-09T21:40:00.000,LIQUOR LICENSE VIOLATION,NaN,NaN,LIQUOR LAW VIOLATION,2018
6,61,06,2018-03-01T14:30:00.000,RETAIL THEFT,NaN,NaN,THEFT,2018
7,18,02,2018-06-01T00:00:00.000,AGG SEX ASSLT OF CHILD FAM MBR,NaN,NaN,OFFENSE INVOLVING CHILDREN,2018
8,67,08B,2018-12-22T04:00:00.000,CHILD ABUSE,NaN,NaN,OFFENSE INVOLVING CHILDREN,2018
9,61,07,2018-12-08T00:00:00.000,AUTOMOBILE,41.80985297,-87.641486912,MOTOR VEHICLE THEFT,2018


In [11]:
srt_gent = start_results.merge(gent_fin, on='community_area')
srt_gent

,community_area,fbi_code,date,description,latitude,longitude,primary_type,year,Neighborhood,1970,1980,1990,2000,2010,1970-2010,Change,Type,Classification,Class_Code
0,28.0,06,2002-05-24T15:00:00.000,OVER $500,41.887111202,-87.641340714,THEFT,2002,Near West Side,-11,-7,-3,-3,11,22,Positive,6 - Gentrification,Gentrification,6
1,28.0,01A,2002-05-14T00:10:00.000,FIRST DEGREE MURDER,41.876059649,-87.68938138,HOMICIDE,2002,Near West Side,-11,-7,-3,-3,11,22,Positive,6 - Gentrification,Gentrification,6
2,28.0,01A,2002-07-06T02:15:00.000,FIRST DEGREE MURDER,41.877529212,-87.688162229,HOMICIDE,2002,Near West Side,-11,-7,-3,-3,11,22,Positive,6 - Gentrification,Gentrification,6
3,28.0,14,2002-04-27T04:05:00.000,TO VEHICLE,NaN,NaN,CRIMINAL DAMAGE,2002,Near West Side,-11,-7,-3,-3,11,22,Positive,6 - Gentrification,Gentrification,6
4,28.0,08B,2002-04-29T17:30:00.000,SIMPLE,NaN,NaN,BATTERY,2002,Near West Side,-11,-7,-3,-3,11,22,Positive,6 - Gentrification,Gentrification,6
5,28.0,08B,2002-04-29T19:18:09.000,SIMPLE,NaN,NaN,BATTERY,2002,Near West Side,-11,-7,-3,-3,11,22,Positive,6 - Gentrification,Gentrification,6
6,28.0,18,2002-05-01T21:24:41.000,POSS: CRACK,NaN,NaN,NARCOTICS,2002,Near West Side,-11,-7,-3,-3,11,22,Positive,6 - Gentrification,Gentrification,6
7,28.0,05,2002-05-07T15:18:00.000,FORCIBLE ENTRY,NaN,NaN,BURGLARY,2002,Near West Side,-11,-7,-3,-3,11,22,Positive,6 - Gentrification,Gentrification,6
8,28.0,18,2002-05-06T22:10:58.000,POSS: CRACK,NaN,NaN,NARCOTICS,2002,Near West Side,-11,-7,-3,-3,11,22,Positive,6 - Gentrification,Gentrification,6
9,28.0,14,2002-05-11T23:45:00.000,TO VEHICLE,NaN,NaN,CRIMINAL DAMAGE,2002,Near West Side,-11,-7,-3,-3,11,22,Positive,6 - Gentrification,Gentrification,6
